In [2]:
!pip install opencv-python

     --------------------------------------- 38.1/38.1 MB 10.7 MB/s eta 0:00:00
     --------------------------------------- 15.8/15.8 MB 14.5 MB/s eta 0:00:00


In [2]:
import cv2

In [11]:
video = cv2.VideoCapture(0)
while video.isOpened():
    success, frame =  video.read()
    if success:
        cv2.imshow('Camera',frame)
        key = cv2.waitKey(1)&0xFF
        if key == 27:  # ESC키(ascii 27) 눌렀을때 
            break
video.release()
cv2.destroyAllWindows()

In [5]:
!pip install matplotlib

     ---------------------------------------- 7.6/7.6 MB 35.0 MB/s eta 0:00:00
     ------------------------------------- 478.0/478.0 kB 31.2 MB/s eta 0:00:00
     ---------------------------------------- 56.1/56.1 kB ? eta 0:00:00
     -------------------------------------- 103.1/103.1 kB 6.2 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 32.1 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 27.3 MB/s eta 0:00:00


In [6]:
import matplotlib.pyplot as plt

In [11]:
import numpy as np
import cv2
import winsound
classes = []
with open('coco.names.txt','r') as f:
  classes =  [line.strip() for line in f.readlines()]
    
colors = np.random.uniform(0,255,size=(len(classes),3))
# 모델 불러오기
yolo_model = cv2.dnn.readNet('yolov3.weights','yolov3.cfg')
layer_names = yolo_model.getLayerNames()
# [200 227 254]
out_layers =  [layer_names[i-1] for i in yolo_model.getUnconnectedOutLayers() ]

def detectVideo(img):    
    height,width,channels = img.shape
    # 입력형식을 yolo형태로 변경 (배치,채널,가로,세로) -> (1, 3, 448, 448)
    blob = cv2.dnn.blobFromImage(img,1.0/256,(448,448),(0,0,0),swapRB=True,crop=False)
    
  # 테스트 영상을 신경망에 입력
    yolo_model.setInput(blob)
  # 출력
    output3 = yolo_model.forward(out_layers)
    class_ids, confidences, boxes = [],[],[]
    # numpy 벡터연산... 파이썬 병렬처리하는 라이브러리...
    for output in output3:  # 3 개
        for vec in output:
            scores =  vec[5:]  # 확률 점수만 획득
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # 확률이 0.5를 넘는 바운딩 박스를 모음
                # 취합한 박스들의 정보중에 중심좌표를 획득
                centerx, centery = int(vec[0]*width),int(vec[1]*height)
                w,h = int(vec[2]*width),int(vec[3]*height)
                x,y = int(centerx-w/2), int(centery-h/2)
                boxes.append([x,y,w,h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    if 0 in class_ids:
        winsound.Beep(frequency=2000,duration=500)
        print('사람 발견')

    # 박스주위에 잔상이나 그런형태의 박스를 배제하기위해서 가장 적합한 확률의 박스만 남긴다
    indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.5,0.4)
    # 살아남은 박스를 영상에 표시
    for i in indexes:
        x,y,w,h = boxes[i]
        text = str(classes[class_ids[i]]) +'%0.3f'%confidences[i]
        cv2.rectangle(img,(x,y),(x+w,y+h),colors[class_ids[i]],2)
        cv2.putText(img,text,(x,y+30),cv2.FONT_HERSHEY_PLAIN,2,colors[class_ids[i]],2)
    return img         



In [12]:
video = cv2.VideoCapture(0)
while video.isOpened():
    success, frame =  video.read()
    if success:
        frame = detectVideo(frame)
        cv2.imshow('Camera',frame)        
        key = cv2.waitKey(1)&0xFF
        if key == 27:  # ESC키(ascii 27) 눌렀을때 
            break
video.release()
cv2.destroyAllWindows()

사람 발견
사람 발견
사람 발견
사람 발견
사람 발견
사람 발견
사람 발견
사람 발견
사람 발견


In [ ]:

  # 구글 코렙에서 사용할때는
  # from google.colab.patches import cv2_imshow
  # cv2_imshow(img)

  # 로컬에서 사용할때
  cv2.imshow("Object detection",img)
  cv2.waitKey(0)
  cv2.destroyAllWindows()

In [8]:
import winsound
winsound.Beep(frequency=2000,duration=500)